In [1]:
#read all files into an array (dataframes)
import numpy as np
import os
import pandas as pd
from IPython.display import display
import datetime
import re
import json
directory = os.getcwd() 

def filter_one_or_zero(n):
  return 1 if (n) else 0 

def split_wares(wares):
  if wares=="{}":
    return None
  res = re.sub(r".*wares\:", "", str(wares))
  return json.loads(res)

def split_inputqueus(inputqueu):
  if (len(inputqueu)==2):
    return dict({"none":0})

  try:
    res = json.loads(inputqueu)
    for i in res.keys():
      res[i] = json.loads(res[i])[2]
    return res
  except:
    return 'error:' + str(inputqueu) 
#  res = json.loads(inputqueu)
#  for i in res.keys():
#    res[i] = json.loads(res[i])[2]
#  return res
      

def read_data_from_files(directory):
  dataframes= []
  for root,dirs,files in os.walk(directory):
      for file in files:
         if (file.endswith("Buildingslogs.csv") & os.path.exists(file)):
            col_names = pd.read_csv(file, nrows=0).columns

            dataframes.append(pd.read_csv(file,sep=";",low_memory=False,header=[0],skiprows=[1]))
  return dataframes


#group the data by building where d[str(serial)] = group

def group_data_by_building(data):
  d = {}
  for data in data :
    for serial, group in data.groupby('serial'):
      d[str(serial)] = group
  return d


def int_to_time(input,hours_mapping):
  return datetime.datetime(2021, 1, 1) + datetime.timedelta(hours=input/hours_mapping)

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +datetime.timedelta(hours=t.minute//30))  

def map_to_real_time(data):
  # hou much is an hour in game time 
  # for example 1hour = 2000 game time unite
  res=data
  hour_in_numbers = (int(data[0]["time"].max()) - int(data[0]["time"].min())) / (365*24)
  for df in res :
    df["time"]=df["time"].apply(int_to_time,args=[hour_in_numbers])
    #df["time"]=df["time"].apply(hour_rounder)
    df.time=df.time.round("0.1S")
  return res  

def read_orders_data_from_files(directory):
  dataframes= []
  for root,dirs,files in os.walk(directory):
      for file in files:
         if (file.endswith("orderslogs.csv") & os.path.exists(file)):
            col_names = pd.read_csv(file, nrows=0).columns

            dataframes.append(pd.read_csv(file,sep=";",low_memory=False,header=[0],skiprows=[1]))
  return dataframes

#get one buildings stockes according to IBP Stock
#data_line is one line where wares != nan of the original data
def data_line_to_one_building_stock_IBP(position):
  #locations = pd.DataFrame(columns=['LOCNO','LOCTYPE','NAME1','NAME2','LAND1','REGIO','ORT01','PSTLZ','PSTL2','STRAS','LOEVM','TIME_ZONE','TELF1','FABKL','KUKLA','NODETYPE','CUST_ATTR1','CUST_ATTR2','CUST_ATTR3','CUST_ATTR4','CUST_ATTR5','CUST_ATTR6','CUST_ATTR7','CUST_ATTR8','CUST_ATTR9','CUST_ATTR10','LONGITUDE','LATITUDE'])
  #stocks = pd.DataFrame(columns=['MATNR','LOC_WERKS','VRFKZ','MNG01'])
  locatins=["0"]*4
  locatins[0]=position
  locatins[1]="V"
  locatins[2]="CET"
  locatins[3]="03"
  #locations["LOCNO"]=position
  #locations["LOCTYPE"]="V"
  #locations["TIME_ZONE"]="CET"
  #locations["FABKL"]="03"
  return locatins
  #group_data_by_building

def write_series_of_location_to_csv_as_IBPlocation(locationseries_nameseries,filename):

  IBPlocations = pd.DataFrame(columns=['LOCID','GEOLATITUDE','GEOLONGITUDE','LOCATIONDEL','LOCATIONREGION','LOCATIONTYPE','LOCCITY','LOCCOUNTRY','LOCNAME','LOCNAME2','LOCTIMEZONE','PHONE','PLANTTYPE','POBOXZIP','PRODUCTIONCALID','STREET','ZIP'])
  IBPlocations[["LOCID","LOCNAME"]]=locationseries_nameseries
  
  #x = set([x[0] for x in test["position"]])
  #locations["'LATITUDE'"]=list(x)
  #IBPlocations["MANDT"]="001"
  IBPlocations["LOCATIONDEL"]=" "
  #IBPlocations["LOGSYS"]="WDL"
  IBPlocations["LOCATIONTYPE"]="P"
  IBPlocations["LOCTIMEZONE"]="CET"
  IBPlocations["PRODUCTIONCALID"]="01"
  IBPlocations["PLANTTYPE"]="PL"
  #IBPlocations.drop_duplicates(subset ="LOCNO",keep = False, inplace = True)
  IBPlocations.to_csv("locations"+filename+ ".csv",sep=";",index=False,header=True)

def write_singel_string_wares_attribute_to_csv_as_IBPmaterial(wares):
  IBPmaterial = pd.DataFrame(columns=['MANDT','LOGSYS','MATNR','LVORM','MATKL','MEINS','PRDHA','MTART','CUST_ATTR1','CUST_ATTR2','CUST_ATTR3','CUST_ATTR4','CUST_ATTR5','CUST_ATTR6','CUST_ATTR7','CUST_ATTR8','CUST_ATTR9','CUST_ATTR10'])

  wares = re.sub(r".*wares\:", "", wares)
  Dict = json.loads(wares)
  wares_amount_data = pd.DataFrame.from_dict(Dict.items())

  wares_amount_data.columns = ['wares', 'amount']
  IBPmaterial["MATNR"]=wares_amount_data["wares"]
  IBPmaterial["MEINS"]="piece"
  IBPmaterial["MANDT"]="001"
  IBPmaterial["LOGSYS"]="WDL"
  IBpmaterialdes = pd.DataFrame(columns=["MANDT","LOGSYS","RESID","SPRAS","KTEXT"])
  IBpmaterialdes["RESID"]=IBPmaterial["MATNR"]
  IBpmaterialdes["SPRAS"]="E"
  IBpmaterialdes["MANDT"]="001"
  IBpmaterialdes["LOGSYS"]="WDL"
  IBpmaterialdes.to_csv("materialsdesc.csv",sep=";",index=False,header=False)
  IBPmaterial.to_csv("materials.csv",sep=";",index=False,header=False)


def write_product_location(serial,wares,input_queues,is_warehouse,is_production_site):
  if is_warehouse:
    IBPproduct_location = pd.DataFrame(columns=["LOCID","PRDID","ASSEMBLYSCRAP","DEMANDSUBSTTYPE","DETPLANNINGSCHEDULINGHORIZON",
    "FAIRSHAREEXCESSSTOCKSEGMENT","FAIRSHAREMAXSTOCKSEGMENT","FAIRSHARESAFETYSTOCKSEGMENT","FORECASTCONSPROFILE",
    "FREEZEHORIZONDISTRIBUTION","FREEZEHORIZONPRODUCTION","GOODSRECEIPTPROCTIME","INBOUNDCALID",
    "LOCATIONPRODUCTDEL","MRPCONTROLLER","PLANDELETIONHORIZON","PLANNEDDELIVERYTIME","PMAXLOTSIZE",
    "PMINLOTSIZE","PPINTEGRATIONMODE","PROCUREMENTTYPE","PRODUCTALLOCATIONPROCEDURE","PROUNDING",
    "SAFETYSTOCKMD","SHIPPINGCALID","SOURCEOFSUPPLYSELPROFILE","SUBNETWORKID"])
    wares=wares
    wares = re.sub(r".*wares\:", "", wares)
    Dict = json.loads(wares)
    wares_amount_data = pd.DataFrame.from_dict(Dict.items())
    wares_amount_data.columns = ['wares', 'amount']
    IBPproduct_location["PRDID"]=wares_amount_data["wares"]
    #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
    IBPproduct_location['LOCID']=str(serial)
    IBPproduct_location["LOCATIONPRODUCTDEL"]=" "
    #IBPproduct_location["VRFKZ"]="X"
    IBPproduct_location.to_csv("product_location.csv",sep=";",index=False,mode='a',header=not os.path.exists("product_location.csv"))
  if is_production_site:
    IBPproduct_location = pd.DataFrame(columns=["LOCID","PRDID","ASSEMBLYSCRAP","DEMANDSUBSTTYPE","DETPLANNINGSCHEDULINGHORIZON",
    "FAIRSHAREEXCESSSTOCKSEGMENT","FAIRSHAREMAXSTOCKSEGMENT","FAIRSHARESAFETYSTOCKSEGMENT","FORECASTCONSPROFILE",
    "FREEZEHORIZONDISTRIBUTION","FREEZEHORIZONPRODUCTION","GOODSRECEIPTPROCTIME","INBOUNDCALID",
    "LOCATIONPRODUCTDEL","MRPCONTROLLER","PLANDELETIONHORIZON","PLANNEDDELIVERYTIME","PMAXLOTSIZE",
    "PMINLOTSIZE","PPINTEGRATIONMODE","PROCUREMENTTYPE","PRODUCTALLOCATIONPROCEDURE","PROUNDING",
    "SAFETYSTOCKMD","SHIPPINGCALID","SOURCEOFSUPPLYSELPROFILE","SUBNETWORKID"])
    wares=input_queues
    Dict = json.loads(wares)
    wares_amount_data=pd.DataFrame(columns=['wares', 'amount'])
    wares_amount_data['wares'] = Dict.keys()
    
    IBPproduct_location["PRDID"]=wares_amount_data["wares"]
    #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
    IBPproduct_location['LOCID']=str(serial)
    IBPproduct_location["LOCATIONPRODUCTDEL"]=" "
    #IBPproduct_location["VRFKZ"]="X"
    IBPproduct_location.to_csv("product_location.csv",sep=";",index=False,mode='a',header=not os.path.exists("product_location.csv"))

def write_singel_string_wares_attribute_to_csv_as_IBPproduct(wares):
  IBPmaterial = pd.DataFrame(columns=['PRDID','PRODDESC','PRODGROUP','PRODTYPE','PRODUCTDEL','PRODUCTHIERARCHY','UOMID'])

  wares = re.sub(r".*wares\:", "", wares)
  Dict = json.loads(wares)
  wares_amount_data = pd.DataFrame.from_dict(Dict.items())

  wares_amount_data.columns = ['wares', 'amount']
  IBPmaterial["PRDID"]=wares_amount_data["wares"]
  IBPmaterial["UOMID"]="ST"
  IBPmaterial['PRODUCTDEL']=" "
  IBPmaterial["PRODDESC"]="no desc lang E"

  #IBpmaterialdes = pd.DataFrame(columns=["MANDT","LOGSYS","RESID","SPRAS","KTEXT"])
  #IBpmaterialdes["RESID"]=IBPmaterial["MATNR"]
  #IBpmaterialdes["SPRAS"]="E"
  #IBpmaterialdes["MANDT"]="001"
  #IBpmaterialdes["LOGSYS"]="WDL"
  #IBpmaterialdes.to_csv("materialsdesc.csv",sep=";",index=False,header=False)
  return IBPmaterial

def write_one_buildings_entry_to_IBPstock(building_entry,filename):
  IBPstocks = pd.DataFrame(columns=['MANDT','DELKZ','LOGSYS','MATNR','LOC_WERKS','LGORT','PLAAB','BLANR','LOC_LIFNR','LOC_KUNNR','CHARG','TST01','VRFKZ','MNG01'])
  wares=building_entry["wares"]
  wares = re.sub(r".*wares\:", "", wares)
  Dict = json.loads(wares)
  wares_amount_data = pd.DataFrame.from_dict(Dict.items())
  wares_amount_data.columns = ['wares', 'amount']
  IBPstocks["MATNR"]=wares_amount_data["wares"]
  IBPstocks["MNG01"]=wares_amount_data["amount"]
  IBPstocks['LOC_WERKS']=str(building_entry["serial"])
  IBPstocks["VRFKZ"]="X"
  IBPstocks['MANDT']="001"
  IBPstocks['LOGSYS']="WDL"
  IBPstocks.to_csv(filename+"stocks.csv",sep=";",index=False,header=False)

def write_one_buildings_orders_to_IBPorders(orders):
  IBPorders = pd.DataFrame(columns=['MANDT','LOGSYS','DELNR','DELPS','DELET','DELKZ','MATNR','LOC_WERKS',
  'CHARG','PLAAB','PLANR','LGORT','LOC_LIFNR','LOC_KUNNR','TST01','VRFKZ','MNG01','MNG02',
  'MNG03','TST02','FIX01','LOC_WERKS_FROM','VERID','DELVR','AUFVR','POSVR',
  'INFNR','EKORG','ESOKZ','MNG04','DEPLOYMENT','MOT_ID','EINVR','DPS_TST','PP_STAGE','REL_FIXED_STATUS'])
  IBPorders['MANDT']="001"
  IBPorders['LOGSYS']="WDL"
  IBPorders['DELNR']=orders.serial.apply(str)+orders.name.apply(str)+orders.time.apply(str)+orders.requested_Ware.apply(str)+orders.amount.apply(str)
  IBPorders.to_csv("IBPorders.csv")

  
def rewrite_wares_to_one_simgle_ware(wares,ware):
  return wares[ware]

def json_and_index(x):
  return json.loads(x)[2]


def write_pds_header(serial,wares,input_queues,is_warehouse,is_production_site):
  if is_warehouse:
    data_to_be_written_to_csv = pd.DataFrame(columns=["LOCID","PRDID","PRODUCTIONVERSION","ASSEMBLYSCRAP"
    ,"BOMEXPLOSIONRULE","OUTPUTCOEFFICIENT","PDSDEL"
    ,"PDSPRIO","PDSVALIDFR","PDSVALIDTO","PMAXLOTSIZE"
    ,"PMINLOTSIZE","PRODUCTIONVERSIONDESCR","PROUNDING"])

    if (wares != "na"):
        wares = re.sub(r".*wares\:", "", wares)
        Dict = json.loads(wares)
        wares_amount_data = pd.DataFrame.from_dict(Dict.items())
        wares_amount_data.columns = ['wares', 'amount']
        data_to_be_written_to_csv["PRDID"]=wares_amount_data["wares"]
        data_to_be_written_to_csv["OUTPUTCOEFFICIENT"]=1
    else: return

    
    #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
    data_to_be_written_to_csv['LOCID']=str(serial)
    data_to_be_written_to_csv["PDSDEL"]=" "
    data_to_be_written_to_csv["PRODUCTIONVERSION"]="01"
    data_to_be_written_to_csv["PDSPRIO"]=1
    data_to_be_written_to_csv["PRODUCTIONVERSIONDESCR"]="come on"
    data_to_be_written_to_csv["PDSVALIDFR"]="2022-08-01 11:12:06.0000000"
    data_to_be_written_to_csv["PDSVALIDTO"]="2023-08-01 11:12:06.0000000"
    

  
    #IBPproduct_location["VRFKZ"]="X"
    data_to_be_written_to_csv.to_csv("pds_header.csv",sep=";",index=False,mode='a',header=not os.path.exists("pds_header.csv"))
  if is_production_site:
    data_to_be_written_to_csv = pd.DataFrame(columns=["LOCID","PRDID","PRODUCTIONVERSION","ASSEMBLYSCRAP"
    ,"BOMEXPLOSIONRULE","OUTPUTCOEFFICIENT","PDSDEL"
    ,"PDSPRIO","PDSVALIDFR","PDSVALIDTO","PMAXLOTSIZE"
    ,"PMINLOTSIZE","PRODUCTIONVERSIONDESCR","PROUNDING"])
    if (len(input_queues)<3):
      return
    wares=input_queues
    Dict = json.loads(wares)
    wares_amount_data = pd.DataFrame.from_dict(Dict.items())
    wares_amount_data.columns = ['wares', 'amount']
    data_to_be_written_to_csv["PRDID"]=wares_amount_data["wares"]

    #print(type(wares_amount_data["amount"].iloc[0]))
    #print(wares_amount_data["amount"])
    data_to_be_written_to_csv["OUTPUTCOEFFICIENT"]=1
    
    data_to_be_written_to_csv["PRDID"]=wares_amount_data["wares"]
    #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
    data_to_be_written_to_csv['LOCID']=str(serial)
    data_to_be_written_to_csv["PDSDEL"]=" "
    data_to_be_written_to_csv["PRODUCTIONVERSION"]="01"
    data_to_be_written_to_csv["PDSPRIO"]=1
    data_to_be_written_to_csv["PRODUCTIONVERSIONDESCR"]="come on"
    #IBPproduct_location["VRFKZ"]="X"
    data_to_be_written_to_csv["PDSVALIDFR"]="2022-08-01 11:12:06.0000000"
    data_to_be_written_to_csv["PDSVALIDTO"]="2023-08-01 11:12:06.0000000"
    #data_to_be_written_to_csv["OUTPUTCOEFFICIENT"]=1
    data_to_be_written_to_csv.to_csv("pds_header.csv",sep=";",index=False,mode='a',header=not os.path.exists("pds_header.csv"))


def read_production_programms_data_from_files(directory):
  dataframes= []
  for root,dirs,files in os.walk(directory):
      for file in files:
         if (file.endswith("productionprograms.csv") & os.path.exists(file)):
            col_names = pd.read_csv(file, nrows=0).columns

            dataframes.append(pd.read_csv(file,sep=";",low_memory=False,header=None,
            names=["serial","name","programm_name","programm_descname","produced_Wares","recruited_workers","consumed_wares_workers","time"]))
  return dataframes


def create_activity_id(x):
  return x["input_wares"]+"+"+x["produced_wares"]+str(x["serial"])

def creat_component_id(x):
  return x["input_wares"]+":"+x["amount"] +"+"+x["produced_wares"]+"C"

def write_pds_comp(serial,producedwares,inputwares,programm_descname):
    data_to_be_written_to_csv = pd.DataFrame(columns=["LOCID","PDSCOMPID","PRDID","PRODUCTIONVERSION","ACTID",
    "COMPONENTCOEFFICIENT","COMPVALIDFR","COMPVALIDTO","PDSCOMPDEL","PDSCOMPITEMNR","PRDFR" ])

    if (producedwares != "{}"):
        wares = re.sub(r".*wares\:", "", producedwares)
        Dict = json.loads(wares)
    # wares_amount_data = pd.DataFrame(columns=['produced_wares','amount'])
        wares_amount_data = pd.DataFrame(list(Dict.items()),columns=['produced_wares','amount'])
        #print(wares_amount_data)
        wares_amount_data = wares_amount_data[['produced_wares']]
        #print(wares_amount_data)
    else: return

    if (inputwares != "{}"):
        wares = re.sub(r".*wares\:", "", inputwares)
        Dict = json.loads(wares)
        #wares_amount_input_data = pd.DataFrame(columns= ['input_wares', 'amount'])
        wares_amount_input_data = pd.DataFrame(list(Dict.items()),columns= ['input_wares', 'amount'])
    else: return

    merged_data=wares_amount_data.merge(wares_amount_input_data,how="cross")

    #print(merged_data)
    merged_data["id"]=range(len(merged_data))
    merged_data["serial"]=serial
    print("fuck",merged_data)
    #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
    data_to_be_written_to_csv["PRDID"]=merged_data["produced_wares"]
    data_to_be_written_to_csv["PRDFR"]=merged_data["input_wares"]
    data_to_be_written_to_csv["COMPONENTCOEFFICIENT"]=merged_data["amount"]
    data_to_be_written_to_csv["PDSCOMPID"]=merged_data.apply(lambda x: creat_component_id(x),axis=1)
    data_to_be_written_to_csv["ACTID"]=merged_data.apply(lambda x: create_activity_id(x),axis=1)
    data_to_be_written_to_csv["PDSCOMPITEMNR"]=merged_data["id"]
    data_to_be_written_to_csv['LOCID']=str(serial)
    data_to_be_written_to_csv["PDSCOMPDEL"]=" "
    data_to_be_written_to_csv["PRODUCTIONVERSION"]="01"

    data_to_be_written_to_csv["COMPVALIDFR"]="2022-08-01 11:12:06.0000000"
    data_to_be_written_to_csv["COMPVALIDTO"]="2023-08-01 11:12:06.0000000"
    #data_to_be_written_to_csv["COMPONENTCOEFFICIENT"]=1
    data_to_be_written_to_csv.to_csv("pds_comp.csv",sep=";",index=False,mode='a',header=not os.path.exists("pds_comp.csv"))

def write_pds_act(inputrow):
    data_to_be_written_to_csv = pd.DataFrame(columns=["ACTID","LOCID","PRDID","PRODUCTIONVERSION","ACTDESCR","ACTDURATION","ACTSEQNR","PDSACTIVITYDEL"])
    #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
    if inputrow["produced_Wares"]=="{}":
      return
    if inputrow["consumed_wares_workers"] == "{}":
      return
    produced_Wares = json.loads(inputrow["produced_Wares"])
    wares_amount_data = pd.DataFrame(list(produced_Wares.items()),columns= ['produced_Wares', 'amount'])
    PDSCOMPID=[]
    inputwares=json.loads(inputrow["consumed_wares_workers"])
    for i in list(inputwares.keys()  ):
      #print(i)
      PDSCOMPID.append(str(i)+"+"+str(wares_amount_data.iloc[0,0])+str(inputrow["serial"]))
    #print(PDSCOMPID[1])
    data_to_be_written_to_csv["ACTID"]=PDSCOMPID
    data_to_be_written_to_csv['ACTSEQNR']=range(len(PDSCOMPID))
    data_to_be_written_to_csv["PRDID"]=list(produced_Wares.keys())[0]
    #data_to_be_written_to_csv["ACTID"]=programm_descname
    data_to_be_written_to_csv['LOCID']=inputrow["serial"]


    data_to_be_written_to_csv["PDSACTIVITYDEL"]=" "
    data_to_be_written_to_csv["PRODUCTIONVERSION"]="01"

    data_to_be_written_to_csv["ACTDESCR"]=inputrow["programm_descname"]
    
    
    #IBPproduct_location["VRFKZ"]="X"
    data_to_be_written_to_csv.to_csv("pds_act.csv",sep=";",index=False,mode='a',header=not os.path.exists("pds_act.csv"))

def write_product_location_from_production_programs(inputrow):
  
  IBPproduct_location = pd.DataFrame(columns=["LOCID","PRDID","ASSEMBLYSCRAP","DEMANDSUBSTTYPE","DETPLANNINGSCHEDULINGHORIZON",
  "FAIRSHAREEXCESSSTOCKSEGMENT","FAIRSHAREMAXSTOCKSEGMENT","FAIRSHARESAFETYSTOCKSEGMENT","FORECASTCONSPROFILE",
  "FREEZEHORIZONDISTRIBUTION","FREEZEHORIZONPRODUCTION","GOODSRECEIPTPROCTIME","INBOUNDCALID",
  "LOCATIONPRODUCTDEL","MRPCONTROLLER","PLANDELETIONHORIZON","PLANNEDDELIVERYTIME","PMAXLOTSIZE",
  "PMINLOTSIZE","PPINTEGRATIONMODE","PROCUREMENTTYPE","PRODUCTALLOCATIONPROCEDURE","PROUNDING",
  "SAFETYSTOCKMD","SHIPPINGCALID","SOURCEOFSUPPLYSELPROFILE","SUBNETWORKID"])
  if (len(inputrow["produced_Wares"])<3):
    return

  IBPproduct_location["PRDID"]=list(json.loads(inputrow["produced_Wares"]).keys())
  #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
  IBPproduct_location['LOCID']=inputrow["serial"]
  IBPproduct_location["LOCATIONPRODUCTDEL"]=" "
  #IBPproduct_location["VRFKZ"]="X"
  print(IBPproduct_location)
  IBPproduct_location.to_csv("product_location_additionals.csv",sep=";",index=False,mode='a',header=not os.path.exists("product_location_additionals.csv"))

def write_pds_header_additionals(inputrow):

    data_to_be_written_to_csv = pd.DataFrame(columns=["LOCID","PRDID","PRODUCTIONVERSION","ASSEMBLYSCRAP"
    ,"BOMEXPLOSIONRULE","OUTPUTCOEFFICIENT","PDSDEL"
    ,"PDSPRIO","PDSVALIDFR","PDSVALIDTO","PMAXLOTSIZE"
    ,"PMINLOTSIZE","PRODUCTIONVERSIONDESCR","PROUNDING"])


    data_to_be_written_to_csv["PRDID"]=list(json.loads(inputrow["produced_Wares"]).keys())
    #IBPproduct_location["MNG01"]=wares_amount_data["amount"]
    data_to_be_written_to_csv['LOCID']=inputrow["serial"]
    data_to_be_written_to_csv["PDSDEL"]=" "
    data_to_be_written_to_csv["PRODUCTIONVERSION"]="01"
    data_to_be_written_to_csv["PDSPRIO"]=1
    data_to_be_written_to_csv["PRODUCTIONVERSIONDESCR"]="come on"
    data_to_be_written_to_csv["PDSVALIDFR"]="2022-08-01 11:12:06.0000000"
    data_to_be_written_to_csv["PDSVALIDTO"]="2023-08-01 11:12:06.0000000"
    data_to_be_written_to_csv["OUTPUTCOEFFICIENT"]=1
    #IBPproduct_location["VRFKZ"]="X"
    data_to_be_written_to_csv.to_csv("pds_header_additionals.csv",sep=";",index=False,mode='a',header=not os.path.exists("pds_header_additionals.csv"))

#ADDLANEID;MOTID;PRDID;LOCFR;LOCTO;GOODSRECEIPTPROCTIME;INBOUNDCALID;LEADTIME;TMAXLOTSIZE;TMINLOTSIZE

def write_pds_transport_lanes(inputrow):
    data_to_be_written_to_csv = pd.DataFrame(columns=["ADDLANEID","LOCFR","LOCTO","MOTID","PRDID","GOODSRECEIPTPROCTIME",
    "INBOUNDCALID","LEADTIME","PURCHASINGORG","REFDOCUMENTCATEGORY","REFDOCUMENTNUMBER","REFDOCUMENTPOSITION",
    "SHIPPINGCALID","SOURCELOCATIONTZ","STOCKTRANSFERTYPE","TLANEBLOCKEDFORPLANNING","TLANEDEL","TLANEPRIO",
    "TLANEVALIDFR","TLANEVALIDTO","TMAXLOTSIZE","TMINLOTSIZE","TRANSPORTCALID","TROUNDING"])
    data_to_be_written_to_csv["LOCFR"]=[inputrow["from_serial"]]
    data_to_be_written_to_csv["LOCTO"]=[inputrow["to_serial"]]
    #data_to_be_written_to_csv["ACTID"]=programm_descname
    data_to_be_written_to_csv["PRDID"]=[inputrow["ware"]]
    data_to_be_written_to_csv["STOCKTRANSFERTYPE"]=0
    data_to_be_written_to_csv["TLANEDEL"]=" "
    data_to_be_written_to_csv["LEADTIME"]=1
    data_to_be_written_to_csv["TLANEPRIO"]=1
    data_to_be_written_to_csv["SOURCELOCATIONTZ"]=" "
    data_to_be_written_to_csv["MOTID"]="road"
    data_to_be_written_to_csv["TRANSPORTCALID"]="01"
    data_to_be_written_to_csv["INBOUNDCALID"]="01"
    data_to_be_written_to_csv["TLANEVALIDFR"]="2022-08-01 11:12:06.0000000"
    data_to_be_written_to_csv["TLANEVALIDTO"]="2023-08-01 11:12:06.0000000"
    data_to_be_written_to_csv["ADDLANEID"]=str(inputrow["from_serial"])+ str(inputrow["to_serial"])
    data_to_be_written_to_csv.to_csv("pds_transport_lanes.csv",sep=";",index=False,mode='a',header=not os.path.exists("pds_transport_lanes.csv"))


def read_transportation_lanes_from_csv(directory):
  dataframes=[]
  for root,dirs,files in os.walk(directory):
      for file in files:
         if (file.endswith("transport_lanes.csv") & os.path.exists(file)):

            dataframes.append(pd.read_csv(file,sep=";",low_memory=False,names=["serial","name","ware","amount","to_serial","to_name","from_serial","from_name"]))
  return dataframes



In [2]:
data = read_data_from_files(directory)


In [3]:
a=map_to_real_time(data)


In [4]:

inputs = a[0].loc[(a[0]["is_production_site"]==1)].query("input_queues.str.len() > 2", engine="python")["input_queues"].apply(split_inputqueus)




In [4]:

products=write_singel_string_wares_attribute_to_csv_as_IBPproduct(a[3].iloc[0]["wares"])
products.to_csv("PRODUCT.csv",sep=";",index=False,header=True)

In [6]:
#write_one_buildings_entry_to_IBPstock(a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ].iloc[0],str(a[0].loc[(a[0]['name'] == "atlanteans_headquarters") ].iloc[0]["serial"]))

In [7]:
a[3]

,serial,name,time,ware_economy_serial,worker_economy_serial,position,reserved_by_worker,workers,is_mine,is_port,...,needs_water_ways,passable,is_production_site,is_warehouse,is_market,ware_priorities,wares,input_queues,produced_Wares,workinpositions
0,914,atlanteans_headquarters,2021-01-01 00:32:52.300,0,1,"(59,53)",0,"number of workers:127,workers:{atlanteans_carr...",0,0,...,0,0,0,1,0,ware_priorities:(),"Numberof wares:117wares:{""log"":""20"",""planks"":""...",na,na,na
1,914,atlanteans_headquarters,2021-01-01 01:05:44.600,0,1,"(59,53)",0,"number of workers:127,workers:{atlanteans_carr...",0,0,...,0,0,0,1,0,ware_priorities:(),"Numberof wares:117wares:{""log"":""20"",""planks"":""...",na,na,na
2,914,atlanteans_headquarters,2021-01-01 01:22:10.700,0,1,"(59,53)",0,"number of workers:127,workers:{atlanteans_carr...",0,0,...,0,0,0,1,0,ware_priorities:(),"Numberof wares:117wares:{""log"":""20"",""planks"":""...",na,na,na
3,914,atlanteans_headquarters,2021-01-01 01:38:36.900,0,1,"(59,53)",0,"number of workers:127,workers:{atlanteans_carr...",0,0,...,0,0,0,1,0,ware_priorities:(),"Numberof wares:117wares:{""log"":""20"",""planks"":""...",na,na,na
4,914,atlanteans_headquarters,2021-01-01 02:11:29.200,0,1,"(59,53)",0,"number of workers:127,workers:{atlanteans_carr...",0,0,...,0,0,0,1,0,ware_priorities:(),"Numberof wares:117wares:{""log"":""20"",""planks"":""...",na,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545901,914,atlanteans_headquarters,2022-01-01 00:14:53.400,8,9,"(59,53)",0,"number of workers:127,workers:{atlanteans_carr...",0,0,...,0,0,0,1,0,ware_priorities:(),"Numberof wares:117wares:{""log"":""50"",""planks"":""...",na,na,na
545902,100216,atlanteans_gold_spinning_mill,2022-01-01 00:20:44.500,8,9,"(60,74)",0,"({100221,65535,atlanteans_carrier})",0,0,...,0,0,1,0,0,na,na,"{""gold"":""[5,5,5]""}",produced_Wares:{},"workinpositions:({atlanteans_carrier:100221,)"
545903,100216,atlanteans_gold_spinning_mill,2022-01-01 00:20:44.500,8,9,"(60,74)",0,"({100221,65535,atlanteans_carrier})",0,0,...,0,0,1,0,0,na,na,"{""gold"":""[5,5,5]""}",produced_Wares:{},"workinpositions:({atlanteans_carrier:100221,)"
545904,100216,atlanteans_gold_spinning_mill,2022-01-01 00:22:23.100,8,9,"(60,74)",0,"({100221,65535,atlanteans_carrier})",0,0,...,0,0,1,0,0,na,na,"{""gold"":""[5,5,5]""}",produced_Wares:{},"workinpositions:({atlanteans_carrier:100221,)"


In [8]:


a[3][["serial","wares","input_queues","is_warehouse","is_production_site"]].drop_duplicates("serial").apply(lambda x: write_product_location(x["serial"],x["wares"],x["input_queues"],x.is_warehouse,x.is_production_site),axis=1)
#a[0].apply(lambda x: test(x["name"],x["serial"]),axis=1)

0         None
373       None
376       None
488       None
735       None
          ... 
519916    None
521200    None
531566    None
540672    None
540720    None
Length: 111, dtype: object

In [31]:


a[3][["serial","wares","input_queues","is_warehouse","is_production_site"]].drop_duplicates("serial").apply(lambda x: write_pds_header(x["serial"],x["wares"],x["input_queues"],x["is_warehouse"],x["is_production_site"]),axis=1)


0         None
373       None
376       None
488       None
735       None
          ... 
519916    None
521200    None
531566    None
540672    None
540720    None
Length: 111, dtype: object

In [10]:
orders=read_orders_data_from_files(directory)
print((orders[0]["time"]))
orders_maped = map_to_real_time(orders)

0           8306
1           8306
2           8306
3           8306
4           8506
          ...   
1927    15863608
1928    15897194
1929    15918214
1930    15951954
1931    15976264
Name: time, Length: 1932, dtype: int64


In [11]:

orders[0].rename(columns={'building_serial':'serial'}, inplace=True)
merged_data = a[0].merge(orders[0], on=["serial","time"], how='left')


In [12]:
print(merged_data[~merged_data["requested_Ware"].isnull()].columns)

Index(['serial', 'name_x', 'time', 'ware_economy_serial',
       'worker_economy_serial', 'position', 'reserved_by_worker', 'workers',
       'is_mine', 'is_port', 'needs_seafaring', 'needs_water_ways', 'passable',
       'is_production_site', 'is_warehouse', 'is_market', 'ware_priorities',
       'wares', 'input_queues', 'produced_Wares', 'workinpositions', 'name_y',
       'requested_Ware', 'amount'],
      dtype='object')


In [27]:
for i in range(len(data)):
  write_series_of_location_to_csv_as_IBPlocation(data[i][["serial","name"]].drop_duplicates(subset="serial"),str(i))




In [14]:
b=data[i][["serial","name"]].drop_duplicates(subset="serial")

In [15]:
write_one_buildings_orders_to_IBPorders(orders[0])

In [16]:
"""
copy= pd.DataFrame(columns=["serial","name","time","wares"])
copy["wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware="log").apply(int).apply(filter_one_or_zero)
#copy.wares= copy.wares.apply(filter_one_or_zero)
copy[["serial","name","time"]] = headquarters[["serial","name","time"]]
#copy.iloc[:,"wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware=i)
sequence =copy.wares.astype(str).str.cat()
#print (sequence)
indices_object = re.finditer(pattern='01', string=sequence)
indices0_1 = [index.start() for index in indices_object]
indices_object = re.finditer(pattern='10', string=sequence)
indices1_0 = [index.start() for index in indices_object]
zeitintervalls =[]
if min(indices1_0) < min(indices0_1):
  indices = sorted(indices1_0.extend(indices0_1))
  if (len(indices)%2 != 0):
    indices.insert(0,0)



copy= pd.DataFrame(columns=["serial","name","time","wares"])
copy["wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware="log")
copy[["serial","name","time"]] = headquarters[["serial","name","time"]]
#copy.iloc[:,"wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware=i)
sequence =copy.wares.astype(str).str.cat()
indices_object = re.finditer(pattern='01', string=sequence)
indices = [index.start() for index in indices_object]

print(indices)
#print(sequence)
#print(sequence.find('6'))




print(indices0_1,indices1_0)


"""

'\ncopy= pd.DataFrame(columns=["serial","name","time","wares"])\ncopy["wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware="log").apply(int).apply(filter_one_or_zero)\n#copy.wares= copy.wares.apply(filter_one_or_zero)\ncopy[["serial","name","time"]] = headquarters[["serial","name","time"]]\n#copy.iloc[:,"wares"]=headquarters["wares"].apply(rewrite_wares_to_one_simgle_ware,ware=i)\nsequence =copy.wares.astype(str).str.cat()\n#print (sequence)\nindices_object = re.finditer(pattern=\'01\', string=sequence)\nindices0_1 = [index.start() for index in indices_object]\nindices_object = re.finditer(pattern=\'10\', string=sequence)\nindices1_0 = [index.start() for index in indices_object]\nzeitintervalls =[]\nif min(indices1_0) < min(indices0_1):\n  indices = sorted(indices1_0.extend(indices0_1))\n  if (len(indices)%2 != 0):\n    indices.insert(0,0)\n\n\n\ncopy= pd.DataFrame(columns=["serial","name","time","wares"])\ncopy["wares"]=headquarters["wares"].apply(rewrite_wares_t

In [5]:
production_programs=read_production_programms_data_from_files(directory)

In [35]:
production_programs[1].apply(lambda x: write_pds_comp(x["serial"],x["produced_Wares"],x["consumed_wares_workers"],x["programm_descname"]),axis=1)

fuck   produced_wares input_wares amount  id  serial
0         planks         log      2   0    2007
fuck   produced_wares input_wares amount  id  serial
0         planks         log      2   0    3273
fuck   produced_wares input_wares amount  id  serial
0    spider_silk        corn      1   0    4386
1    spider_silk       water      1   1    4386
fuck      produced_wares      input_wares amount  id  serial
0  atlanteans_bread            water      2   0    4729
1  atlanteans_bread  blackroot_flour      1   1    4729
2  atlanteans_bread         cornmeal      1   2    4729
fuck   produced_wares input_wares amount  id  serial
0    smoked_fish        fish      2   0    4763
1    smoked_fish         log      1   1    4763
fuck   produced_wares input_wares amount  id  serial
0    smoked_fish        fish      2   0    4763
1    smoked_fish         log      1   1    4763
fuck   produced_wares input_wares amount  id  serial
0    smoked_meat        meat      2   0    4763
1    smoked_meat     

0      None
1      None
2      None
3      None
4      None
       ... 
274    None
275    None
276    None
277    None
278    None
Length: 279, dtype: object

In [20]:
test=production_programs[1].loc[101]

In [21]:
producedwares= test["produced_Wares"]
serial = test["serial"]
programm_descname = test["programm_descname"]
programm_name= test["programm_name"]


In [22]:
def filter_double_products(x):
  if "/" in x["PRDID"].iloc[0]:
    splited = x["PRDID"].iloc[0].rpartition("/")
    #print(splited[2])
    newrow=x
    #print(newrow)
    x.loc["PRDID",0]=splited[0]
    newrow.loc["PRDID",0]=splited[2]
    #print(x.append(newrow,ignore_index = True))
    #print(x)
    #print(newrow)
    return x.concat(newrow,axis=1)
  return 


In [34]:
production_programs[1].apply(lambda inputrow:write_pds_act(inputrow),axis=1)




0      None
1      None
2      None
3      None
4      None
       ... 
274    None
275    None
276    None
277    None
278    None
Length: 279, dtype: object

In [24]:
a = pd.read_csv("pds_comp.csv",sep=";")
print(len(a))
print(len(a.drop_duplicates("PDSCOMPID")))

260
84


In [33]:
#additionals
production_programs[1].apply(lambda inputrow:write_product_location_from_production_programs(inputrow),axis=1)
production_programs[1].apply(lambda inputrow:write_pds_header_additionals(inputrow),axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
274    None
275    None
276    None
277    None
278    None
Length: 279, dtype: object

In [10]:
trans_lanes=read_transportation_lanes_from_csv(directory)

In [21]:
trans_lanes[1]

,serial,name,ware,amount,to_serial,to_name,from_serial,from_name
0,1141,constructionsite,atlanteans_builder,1,1091,constructionsite,1141,constructionsite
1,1219,constructionsite,atlanteans_builder,1,914,atlanteans_headquarters,1219,constructionsite
2,1219,constructionsite,planks,1,914,atlanteans_headquarters,1219,constructionsite
3,1219,constructionsite,log,2,914,atlanteans_headquarters,1219,constructionsite
4,1219,constructionsite,log,2,914,atlanteans_headquarters,1219,constructionsite
...,...,...,...,...,...,...,...,...
8395,46294,atlanteans_smelting_works,coal,2,20936,atlanteans_woodcutters_house,46294,atlanteans_smelting_works
8396,46294,atlanteans_smelting_works,coal,2,914,atlanteans_headquarters,46294,atlanteans_smelting_works
8397,2007,atlanteans_sawmill,log,1,10235,atlanteans_foresters_house,2007,atlanteans_sawmill
8398,15555,atlanteans_dungeon,trident_heavy_double,1,15109,atlanteans_barracks,15555,atlanteans_dungeon


In [23]:
locations=a[3][["serial"]].drop_duplicates()

In [27]:
locations=locations.merge(locations,how="cross").merge(products,how="cross")

In [41]:
locations.rename({'serial_x': 'to_serial', 'serial_y': 'from_serial','PRDID':'ware'}, axis=1, inplace=True)

In [43]:
locations

,to_serial,from_serial,ware,PRODDESC,PRODGROUP,PRODTYPE,PRODUCTDEL,PRODUCTHIERARCHY,UOMID
0,914,914,log,no desc lang E,NaN,NaN,,NaN,ST
1,914,914,planks,no desc lang E,NaN,NaN,,NaN,ST
2,914,914,spidercloth,no desc lang E,NaN,NaN,,NaN,ST
3,914,914,granite,no desc lang E,NaN,NaN,,NaN,ST
4,914,914,spider_silk,no desc lang E,NaN,NaN,,NaN,ST
...,...,...,...,...,...,...,...,...,...
1441552,104082,104082,vest_padded,no desc lang E,NaN,NaN,,NaN,ST
1441553,104082,104082,protector_padded,no desc lang E,NaN,NaN,,NaN,ST
1441554,104082,104082,boots_sturdy,no desc lang E,NaN,NaN,,NaN,ST
1441555,104082,104082,boots_swift,no desc lang E,NaN,NaN,,NaN,ST


In [47]:
data_to_be_written_to_csv = pd.DataFrame(columns=["ADDLANEID","LOCFR","LOCTO","MOTID","PRDID","GOODSRECEIPTPROCTIME",
"INBOUNDCALID","LEADTIME","PURCHASINGORG","REFDOCUMENTCATEGORY","REFDOCUMENTNUMBER","REFDOCUMENTPOSITION",
"SHIPPINGCALID","SOURCELOCATIONTZ","STOCKTRANSFERTYPE","TLANEBLOCKEDFORPLANNING","TLANEDEL","TLANEPRIO",
"TLANEVALIDFR","TLANEVALIDTO","TMAXLOTSIZE","TMINLOTSIZE","TRANSPORTCALID","TROUNDING"])
data_to_be_written_to_csv["LOCFR"]=locations["from_serial"]
data_to_be_written_to_csv["LOCTO"]=locations["to_serial"]
#data_to_be_written_to_csv["ACTID"]=programm_descname
data_to_be_written_to_csv["PRDID"]=locations["ware"]
data_to_be_written_to_csv["STOCKTRANSFERTYPE"]=0
data_to_be_written_to_csv["TLANEDEL"]=" "
data_to_be_written_to_csv["LEADTIME"]=1
data_to_be_written_to_csv["TLANEPRIO"]=1
data_to_be_written_to_csv["SOURCELOCATIONTZ"]=" "
data_to_be_written_to_csv["MOTID"]="road"
data_to_be_written_to_csv["TRANSPORTCALID"]="01"
data_to_be_written_to_csv["INBOUNDCALID"]="01"
data_to_be_written_to_csv["TLANEVALIDFR"]="2022-08-01 11:12:06.0000000"
data_to_be_written_to_csv["TLANEVALIDTO"]="2023-08-01 11:12:06.0000000"
data_to_be_written_to_csv["ADDLANEID"]=range(len(locations))

In [49]:
data_to_be_written_to_csv.to_csv("transportation_lanes.csv",header=True,index=False,sep=";")

In [44]:
locations.apply(lambda inputrow:write_pds_transport_lanes(inputrow),axis=1)

KeyboardInterrupt: 

In [36]:
trans_lanes[1].loc[trans_lanes[1]['ware'].isin(products.PRDID.to_list())].drop_duplicates(["ware","from_serial","to_serial"]).apply(lambda inputrow:write_pds_transport_lanes(inputrow),axis=1)

2       None
3       None
6       None
7       None
13      None
        ... 
8381    None
8382    None
8383    None
8391    None
8394    None
Length: 1147, dtype: object